In [65]:
import sys
sys.path.append('../model/dataset.py')
sys.path.append('..')
from model import dataset
import cv2
import numpy as np
def show_image(image):
    image = image.astype(np.uint8)
    cv2.imshow('img', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image = cv2.imread('../datasets/frames/frame_5660.jpg')
image = cv2.rectangle(image,(0,0),(180,1080),(0,255,0), thickness=-1)
image = cv2.rectangle(image,(1920-180, 0),(1920,1080),(0,255,0), thickness=-1)
image = cv2.rectangle(image,(0, 980),(1920,1080),(0,255,0), thickness=-1)
pts1 = np.array([[0,0],[0,700],[650, 0]], np.int32)
pts2 = np.array([[1920,0],[1920,700],[1920 - 650, 0]], np.int32)

image = cv2.fillPoly(image,[pts1],(0,255,255))
image = cv2.fillPoly(image,[pts2],(0,255,255))



# show_image(image)
cv2.imwrite("test.jpg", image)

True

True